### Define model and cost

In [ ]:
import numpy as np
import casadi as ca


# cartpole class
class CartPole:
    def __init__(self, module='casadi'):
        if module == 'casadi':
            self.sin = ca.sin
            self.cos = ca.cos
            self.pi = ca.pi
        # else if module == 'numpy':
        #     self.sin = np.sin
        #     self.cos = np.cos
        #     self.pi = np.pi
        # else:
        #     raise TypeError

        self.mc = 2.0
        self.mp = 0.2
        self.l = 0.5
        self.gc = 9.80665

        self.lbu = -15.0
        self.ubu = 15.0
        self.lby = 1.0
        self.uby = 1.0

    def dynamics(self, x, u):
        sin = self.sin
        cos = self.cos
        pi = self.pi

        mc = self.mc
        mp = self.mp
        l = self.l
        gc = self.gc

        y = x[0]
        th = x[1]
        dy = x[2]
        dth = x[3]
        f = u[0]

        ddy = (f+mp*sin(th)*(l*dth*dth+gc*cos(th))) / (mc+mp*sin(th)*sin(th)) 
        ddth = (-f*cos(th)-mp*l*dth*dth*cos(th)*sin(th)-(mc+mp)*gc*sin(th)) / (l * (mc+mp*sin(th)*sin(th))) 
        return np.array([dy, dth, ddy, ddth])


In [ ]:
# cost calculating class
class CostFunction:
    def __init__(self, module='casadi'):
        if module == 'casadi':
            self.sin = ca.sin
            self.cos = ca.cos
            self.pi = ca.pi
        # else if module == 'numpy':
        #     self.sin = np.sin
        #     self.cos = np.cos
        #     self.pi = np.pi
        # else:
        #     raise TypeError
        pi = self.pi

        self.n_x = 4
        self.n_u = 1
        self.x_ref = np.array([0.0, pi, 0.0, 0.0])
        self.Q = np.array([2.5, 10.0, 0.01, 0.01])
        self.R = np.array([1.0])
        self.Q_f = np.array([2.5, 10.0, 0.01, 0.01])

    def stage_cost(self, x, u):
        l = 0.0
        for i in range(self.n_x):
            l += 0.5 * self.Q[i] * (x[i] - self.x_ref[i]) ** 2
        for i in range(self.n_u):
            l += 0.5 * self.R[i] * u[i]**2
        return l
    
    def terminal_cost(self, x):
        lf = 0.0
        for i in range(self.n_x):
            lf += 0.5 * self.Q_f[i] * (x[i] - self.x_ref[i]) ** 2
        return lf



In [ ]:
cartpole = CartPole()
cost = CostFunction()


### Approximate optimal control problem and formulate it as NLP

In [ ]:
# dimension of state and input
n_x = 4
n_u = 1

# horizon length[s], total grids
T = 5.0
N = 200
dt = T / N



In [ ]:
# empty NLP

# objective function
J = 0.0
# decision variables: x0, u0, x1, ...
w = []
# constrints
g = []

# initial guess of w
w0 = []
# bound for variable
lbw = []
ubw = []
# bound for constrints
lbg = []
ubg = []


In [ ]:
# initial state
Xk = ca.MX.sym('X0', n_x)
w += [Xk]

w0 += [0.0] * n_x
lbw += [0.0] * n_x
ubw += [0.0] * n_x

# stage k
for k in range(N):
    # input u
    Uk = ca.MX.sym(f'X_{k}', n_u)
    w += [Uk]
    w0 += [0.0]
    lbw += [-15.0]
    ubw += [15.0]

    # cost function
    J += cost.stage_cost(Xk, Uk) * dt

    # next state
    Xk1 = ca.MX.sym(f'X_{k + 1}', n_x)
    w += [Xk1]
    w0 += [0.0] * n_x
    lbw += [-ca.inf] * n_x
    ubw += [ca.inf] * n_x

    # dynamics as equality constraint (close gaps)
    dXk = ca.vertcat(*cartpole.dynamics(Xk, Uk))
    Xk_rollout = Xk + dXk * dt
    g += [Xk_rollout - Xk1]
    lbg += [0] * n_x
    ubg += [0] * n_x

    # k = k + 1
    Xk = Xk1

# terminal cost
J += cost.terminal_cost(Xk)


In [ ]:
nlp = {'f': J, 'x': ca.vertcat(*w), 'g': ca.vertcat(*g)}


### Choose solver and solve NLP

In [ ]:
# use IPOPT as NLP solver
solver_option = {'ipopt': {'mu_min': 0.001, 'max_iter': 1000}}
solver = ca.nlpsol('solver', 'ipopt', nlp, solver_option)

# solve
sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)


### Plot trajectory

In [ ]:
import path_ocpy
from ocpy.plotter import Plotter


w_opt = sol['x'].full().reshape(-1)

xs_opt = np.zeros((N + 1, n_x))
us_opt = np.zeros((N, n_u))
ts = np.array([i * dt for i in range(N + 1)])

for i in range(N + 1):
    n = n_x + n_u
    xs_opt[i] = w_opt[i * n: i * n + n_x]

for i in range(N):
    n = n_x + n_u
    us_opt[i] = w_opt[i * n + n_x: (i + 1) * n]

log_dir = path_ocpy.dir + '../log/cartpole'

# print('final cost:', sol.value(J))
print('final cost:', sol['f'])

plotter = Plotter(log_dir, xs_opt, us_opt, ts)
plotter.plot(save=False)
